<a href="https://colab.research.google.com/github/surajjorwar/AirBnb-booking-analysis---python/blob/main/Book_Recommendation_System.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [33]:
#installing fuzzywuzzy library
!pip install fuzzywuzzy

In [34]:
#importing library's
import numpy as np
import pandas as pd
from fuzzywuzzy import fuzz, process
from IPython.display import display, Image, HTML

In [35]:
#loading csv files
book = pd.read_csv('/content/drive/MyDrive/BOOK_RECOMMENDER_SYSTEM/Copy of Books.csv')
rating = pd.read_csv('/content/drive/MyDrive/BOOK_RECOMMENDER_SYSTEM/Copy of Ratings.csv')
users = pd.read_csv('/content/drive/MyDrive/BOOK_RECOMMENDER_SYSTEM/Copy of Users.csv')

<ipython-input-35-b36acff5a32a>:2: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  book = pd.read_csv('/content/drive/MyDrive/BOOK_RECOMMENDER_SYSTEM/Copy of Books.csv')


# **EXPLORING THE DATA **

In [36]:
book.head(2)

,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L
0,0195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...
1,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...


In [37]:
book.shape

(271360, 8)

In [38]:
book.isna().sum()

ISBN                   0
Book-Title             0
Book-Author            1
Year-Of-Publication    0
Publisher              2
Image-URL-S            0
Image-URL-M            0
Image-URL-L            3
dtype: int64

In [39]:
#droping null values
book.dropna(inplace = True)

In [40]:
book.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 271354 entries, 0 to 271359
Data columns (total 8 columns):
 #   Column               Non-Null Count   Dtype 
---  ------               --------------   ----- 
 0   ISBN                 271354 non-null  object
 1   Book-Title           271354 non-null  object
 2   Book-Author          271354 non-null  object
 3   Year-Of-Publication  271354 non-null  object
 4   Publisher            271354 non-null  object
 5   Image-URL-S          271354 non-null  object
 6   Image-URL-M          271354 non-null  object
 7   Image-URL-L          271354 non-null  object
dtypes: object(8)
memory usage: 18.6+ MB


In [41]:
# droping duplicate values if any
book.drop_duplicates(inplace = True )
book.shape

(271354, 8)

In [42]:
#exploring rating dataframe
rating.head(2)

,User-ID,ISBN,Book-Rating
0,276725,034545104X,0
1,276726,0155061224,5


In [43]:
rating.shape

(1149780, 3)

In [44]:
#checking null values
rating.isnull().sum()

User-ID        0
ISBN           0
Book-Rating    0
dtype: int64

In [45]:
# droping duplicate values if any
rating.drop_duplicates(inplace = True )
rating.shape


(1149780, 3)

In [46]:
#Exploring user data frame
users.head(2)

,User-ID,Location,Age
0,1,"nyc, new york, usa",NaN
1,2,"stockton, california, usa",18.0


In [47]:
# checing the size of data frame
users.shape

(278858, 3)

In [48]:
#checking null values
users.isnull().sum()

User-ID          0
Location         0
Age         110762
dtype: int64

In [49]:
users['Age'].fillna(users['Age'].mean(),inplace = True)

In [50]:
# droping duplicate values if any
users.drop_duplicates(inplace = True )
users.shape

(278858, 3)